In [1]:
import json
import os
import igraph as ig
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

from typing import Dict, List, Tuple, Optional, Set
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from pydantic import BaseModel,Field
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_community.tools import TavilySearchResults
from ER_extraction import entities_out
from langchain_deepseek import ChatDeepSeek

load_dotenv()
# Groq api key define
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
llm = ChatGroq(model="llama-3.3-70b-versatile")

# OPENAI api key define
# os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
# model = ChatOpenAI(model="gpt-4o",base_url=os.getenv("base_url"))

# Deepseek api key define
os.environ["DEEPSEEK_API_KEY"] = os.getenv("DEEPSEEK_API_KEY")
model = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

# Web search api define
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

In [2]:
data = entities_out()
print(data)

{'entities': [{'entity_name': 'milk', 'entity_type': '', 'description': '', 'source_id': ''}, {'entity_name': ' butter', 'entity_type': '', 'description': '', 'source_id': ''}, {'entity_name': ' ghee', 'entity_type': '', 'description': '', 'source_id': ''}, {'entity_name': ' yoghurt', 'entity_type': '', 'description': '', 'source_id': ''}, {'entity_name': ' buttermilk', 'entity_type': '', 'description': '', 'source_id': ''}, {'entity_name': ' cheese', 'entity_type': '', 'description': '', 'source_id': ''}, {'entity_name': ' cattle', 'entity_type': '', 'description': '', 'source_id': ''}, {'entity_name': ' sheep', 'entity_type': '', 'description': '', 'source_id': ''}, {'entity_name': ' goat', 'entity_type': '', 'description': '', 'source_id': ''}, {'entity_name': ' yak', 'entity_type': '', 'description': '', 'source_id': ''}, {'entity_name': ' buffalo', 'entity_type': '', 'description': '', 'source_id': ''}, {'entity_name': ' mozzarella', 'entity_type': '', 'description': '', 'source_i

# Knowledge Graph Processing

In [3]:
def load_knowledge_graph(json_data: Dict) -> Tuple[List[str], List[Tuple[int, int, dict]]]:
    """
    Load knowledge graph data from JSON and prepare it for igraph conversion.

    Args:
        json_data: Dictionary containing entities and relationships

    Returns:
        Tuple containing:
        - List of unique entity names
        - List of edges with attributes
    """
    # Extract entities and create a mapping of names to indices
    entities = [entity['entity_name'].strip() for entity in json_data['entities']]
    entity_to_idx = {name: idx for idx, name in enumerate(entities)}

    # Process relationships into edge list with attributes
    edges = []
    for rel in json_data['relationships']:
        src = rel['src_id'].strip()
        tgt = rel['tgt_id'].strip()

        # Only add edge if both nodes exist
        if src in entity_to_idx and tgt in entity_to_idx:
            edge_attrs = {
                'description': rel['description'],
                'weight': float(rel['weight']) if rel['weight'] else 1.0
            }
            edges.append((entity_to_idx[src], entity_to_idx[tgt], edge_attrs))

    return entities, edges


def create_igraph(vertices: List[str], edges: List[Tuple[int, int, dict]]) -> ig.Graph:
    """
    Create an igraph Graph object from vertices and edges.

    Args:
        vertices: List of vertex names
        edges: List of tuples (source_idx, target_idx, attributes)

    Returns:
        igraph.Graph object
    """
    # Create directed graph
    g = ig.Graph(directed=True)

    # Add vertices
    g.add_vertices(len(vertices))
    g.vs['name'] = vertices

    # Add edges with attributes
    if edges:  # Check if there are any edges
        edge_tuples = [(e[0], e[1]) for e in edges]
        g.add_edges(edge_tuples)

        # Add edge attributes
        for idx, (_, _, attrs) in enumerate(edges):
            for key, value in attrs.items():
                if g.es.attribute_names().count(key) == 0:
                    g.es[key] = [None] * len(g.es)
                g.es[idx][key] = value

    return g


def personalized_pagerank_search(g: ig.Graph, query_vertices: List[str], damping: float = 0.85) -> Dict[str, float]:
    """
    Perform personalized PageRank search on the graph.

    Args:
        g: igraph Graph object
        query_vertices: List of vertex names to use as personalization vector
        damping: Damping factor for PageRank (default: 0.85)

    Returns:
        Dictionary mapping vertex names to their PageRank scores
    """
    # Create personalization vector by finding vertex indices
    reset_vertices = []
    for vertex in query_vertices:
        try:
            vertex_idx = g.vs.find(name=vertex).index
            reset_vertices.append(vertex_idx)
        except ValueError:
            continue

    # If no valid vertices found, use uniform distribution
    if not reset_vertices:
        return {v['name']: 1.0 / len(g.vs) for v in g.vs}

    # Calculate personalized PageRank
    weights = g.es.get_attribute_values('weight') if 'weight' in g.es.attributes() else None

    pagerank_scores = g.personalized_pagerank(
        weights=weights,
        damping=damping,
        reset_vertices=reset_vertices
    )

    # Create results dictionary
    results = {v['name']: score for v, score in zip(g.vs, pagerank_scores)}
    return dict(sorted(results.items(), key=lambda x: x[1], reverse=True))


def process_knowledge_graph(json_data: Dict, query_entities: List[str]) -> Dict[str, float]:
    """
    Process knowledge graph and perform personalized PageRank search.

    Args:
        json_data: Knowledge graph data in JSON format
        query_entities: List of entity names to search for

    Returns:
        Dictionary of entity names and their PageRank scores
    """
    # Convert JSON to igraph structure
    vertices, edges = load_knowledge_graph(json_data)
    graph = create_igraph(vertices, edges)

    # Perform personalized PageRank search
    results = personalized_pagerank_search(graph, query_entities)

    return results

# Improve Knowledge Graph Processing with similarity search

In [4]:
def load_knowledge_graph_with_embeddings(json_data: Dict) -> Tuple[List[str], List[Tuple[int, int, dict]], np.ndarray]:
    """
    Load knowledge graph data with entity embeddings for similarity calculations.
    """
    entities, edges = load_knowledge_graph(json_data)

    # Generate embeddings for entities
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model.encode(entities)

    return entities, edges, embeddings


def similarity_weighted_pagerank(g: ig.Graph,
                                 query_vertices: List[str],
                                 embeddings: np.ndarray,
                                 damping: float = 0.85,
                                 similarity_weight: float = 0.5) -> Dict[str, float]:
    """
    Combine PersonalizedPageRank with cosine similarity for ranking.
    """
    # Get embeddings for query vertices
    query_indices = []
    for vertex in query_vertices:
        try:
            idx = g.vs.find(name=vertex).index
            query_indices.append(idx)
        except ValueError:
            continue

    if not query_indices:
        return {v['name']: 1.0 / len(g.vs) for v in g.vs}

    # Calculate cosine similarities
    query_embedding = np.mean(embeddings[query_indices], axis=0).reshape(1, -1)
    similarities = cosine_similarity(embeddings, query_embedding).flatten()

    # Calculate PageRank scores
    weights = g.es.get_attribute_values('weight') if 'weight' in g.es.attributes() else None
    pagerank_scores = g.personalized_pagerank(
        weights=weights,
        damping=damping,
        reset_vertices=query_indices
    )

    # Combine scores
    combined_scores = {}
    for v, pr_score, sim_score in zip(g.vs, pagerank_scores, similarities):
        combined_scores[v['name']] = (1 - similarity_weight) * pr_score + similarity_weight * sim_score

    return dict(sorted(combined_scores.items(), key=lambda x: x[1], reverse=True))


def process_knowledge_graph_with_similarity(json_data: Dict,
                                            query_entities: List[str],
                                            similarity_weight: float = 0.5) -> Dict[str, float]:
    """
    Process knowledge graph with combined PageRank and similarity scoring.
    """
    vertices, edges, embeddings = load_knowledge_graph_with_embeddings(json_data)
    graph = create_igraph(vertices, edges)
    results = similarity_weighted_pagerank(
        graph,
        query_entities,
        embeddings,
        similarity_weight=similarity_weight
    )
    return results

# Improve Knowladge graph search with chain-of-thought

In [5]:
class ChainOfThoughtGraphSearch:
    def __init__(self, json_data: Dict, model_name: str = 'all-MiniLM-L6-v2'):
        self.model = SentenceTransformer(model_name)
        self.vertices, self.edges, self.embeddings = load_knowledge_graph_with_embeddings(json_data)
        self.graph = create_igraph(self.vertices, self.edges)
        self.reasoning_history: List[Dict] = []

    def iterative_search(self, query: str, max_steps: int = 3) -> Dict[str, float]:
        current_entities = set(self._initial_entity_extraction(query))
        all_relevant_entities: Set[str] = set()

        for step in range(max_steps):
            # Get current results
            step_results = similarity_weighted_pagerank(
                self.graph,
                list(current_entities),
                self.embeddings
            )

            # Update reasoning history
            self.reasoning_history.append({
                'step': step,
                'query_entities': current_entities,
                'top_results': dict(list(step_results.items())[:5])
            })

            # Expand search with new entities
            new_entities = self._extract_related_entities(step_results, threshold=0.3)
            all_relevant_entities.update(current_entities)
            current_entities = new_entities - all_relevant_entities

            if not current_entities:
                break

        return self._aggregate_results()

    def _initial_entity_extraction(self, query: str) -> List[str]:
        query_embedding = self.model.encode([query])[0]
        similarities = cosine_similarity([query_embedding], self.embeddings)[0]
        return [self.vertices[i] for i in np.argsort(similarities)[-3:]]

    def _extract_related_entities(self, results: Dict[str, float], threshold: float) -> Set[str]:
        return {entity for entity, score in results.items() if score > threshold}

    def _aggregate_results(self) -> Dict[str, float]:
        all_scores = {}
        decay_factor = 0.8

        for step, history in enumerate(self.reasoning_history):
            step_weight = decay_factor ** step
            for entity, score in history['top_results'].items():
                all_scores[entity] = all_scores.get(entity, 0) + score * step_weight

        return dict(sorted(all_scores.items(), key=lambda x: x[1], reverse=True))

    def get_reasoning_chain(self) -> List[Dict]:
        return self.reasoning_history

# Visualization

In [6]:
def create_plotly_graph_visualization(
        g: ig.Graph,
        title: str,
        pagerank_scores: Optional[Dict[str, float]] = None,
        highlight_vertices: Optional[List[str]] = None,
        edge_threshold: float = 0.0,
        reasoning_step: Optional[int] = None,
        embeddings: Optional[np.ndarray] = None
) -> go.Figure:
    """
    Create an interactive plotly visualization of the graph with embedding-based layout.

    Args:
        g: igraph Graph object
        title: Title for the visualization
        pagerank_scores: Optional dictionary of vertex names to PageRank scores
        highlight_vertices: Optional list of vertex names to highlight
        edge_threshold: Minimum PageRank score for vertices to include
        reasoning_step: Optional step number in the reasoning chain
        embeddings: Optional numpy array of entity embeddings for layout

    Returns:
        plotly Figure object
    """
    # Use UMAP or PCA layout if embeddings are provided, otherwise use Kamada-Kawai
    if embeddings is not None:
        from umap import UMAP
        layout = UMAP(n_components=2, random_state=42).fit_transform(embeddings)
    else:
        layout = g.layout_kamada_kawai()

    # Calculate color scheme based on scores
    if pagerank_scores:
        scores = np.array([pagerank_scores.get(v['name'], 0) for v in g.vs])
        normalized_scores = (scores - scores.min()) / (scores.max() - scores.min())
        colorscale = px.colors.sequential.Viridis
        vertex_colors = [px.colors.sample_colorscale(colorscale, score)[0]
                         for score in normalized_scores]
    else:
        vertex_colors = ['lightblue'] * len(g.vs)

    # Adjust vertex sizes based on scores and highlighting
    vertex_sizes = [30] * len(g.vs)
    if pagerank_scores:
        max_score = max(pagerank_scores.values())
        for i, vertex in enumerate(g.vs):
            score = pagerank_scores.get(vertex['name'], 0)
            vertex_sizes[i] = 20 + (score / max_score) * 50

            if highlight_vertices and vertex['name'] in highlight_vertices:
                vertex_colors[i] = 'red'
                vertex_sizes[i] *= 1.5

    # Create edge traces with improved styling
    edge_traces = []
    for edge in g.es:
        source = g.vs[edge.source]
        target = g.vs[edge.target]

        if pagerank_scores:
            source_score = pagerank_scores.get(source['name'], 0)
            target_score = pagerank_scores.get(target['name'], 0)
            if source_score < edge_threshold or target_score < edge_threshold:
                continue

        x0, y0 = layout[edge.source]
        x1, y1 = layout[edge.target]

        # Calculate edge width based on weight
        weight = edge['weight'] if 'weight' in g.es.attributes() else 1.0
        edge_width = 0.5 + weight * 2

        # Correctly access edge description attribute
        description = edge['description'] if 'description' in g.es.attributes() else ''

        edge_trace = go.Scatter(
            x=[x0, x1, None],
            y=[y0, y1, None],
            mode='lines',
            line=dict(width=edge_width, color='#888'),
            hoverinfo='text',
            text=description
        )
        edge_traces.append(edge_trace)

    # Create figure
    fig = go.Figure()

    # Add edges
    for trace in edge_traces:
        fig.add_trace(trace)

    # Add nodes with improved styling
    node_trace = go.Scatter(
        x=[layout[i][0] for i in range(len(g.vs))],
        y=[layout[i][1] for i in range(len(g.vs))],
        mode='markers+text',
        marker=dict(
            size=vertex_sizes,
            color=vertex_colors,
            line=dict(width=2, color='black'),
            symbol='circle'
        ),
        text=[f"{v['name']}<br>Score: {pagerank_scores.get(v['name'], 0):.4f}"
              if pagerank_scores else v['name'] for v in g.vs],
        textposition="top center",
        hoverinfo='text'
    )
    fig.add_trace(node_trace)

    # Update layout with reasoning step information
    title_text = f"{title}<br>Step {reasoning_step}" if reasoning_step is not None else title
    fig.update_layout(
        title=dict(
            text=title_text,
            x=0.5,
            y=0.95
        ),
        showlegend=False,
        hovermode='closest',
        margin=dict(b=20, l=5, r=5, t=60),
        plot_bgcolor='white',
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
    )

    return fig


def create_reasoning_chain_visualization(
        search_instance: 'ChainOfThoughtGraphSearch',
        score_threshold: float = 0.01
) -> List[go.Figure]:
    """
    Create visualizations for each step in the reasoning chain.

    Args:
        search_instance: ChainOfThoughtGraphSearch instance
        score_threshold: Threshold for including vertices

    Returns:
        List of plotly figures, one for each reasoning step
    """
    figures = []

    for step, history in enumerate(search_instance.reasoning_history):
        step_fig = create_plotly_graph_visualization(
            search_instance.graph,
            f"Knowledge Graph Exploration - Step {step + 1}",
            history['top_results'],
            list(history['query_entities']),
            score_threshold,
            step + 1,
            search_instance.embeddings
        )
        figures.append(step_fig)

    return figures


def display_graph_visualizations(
        search_instance: 'ChainOfThoughtGraphSearch',
        score_threshold: float = 0.01
) -> Tuple[go.Figure, List[go.Figure]]:
    """
    Process knowledge graph and create full visualization plus reasoning chain.

    Args:
        search_instance: ChainOfThoughtGraphSearch instance
        score_threshold: Threshold for including vertices

    Returns:
        Tuple of (final_graph_figure, list_of_reasoning_step_figures)
    """
    # Create final aggregate visualization
    final_results = search_instance._aggregate_results()
    final_graph = create_plotly_graph_visualization(
        search_instance.graph,
        "Final Knowledge Graph Results",
        final_results,
        list(search_instance.reasoning_history[-1]['query_entities']),
        score_threshold,
        embeddings=search_instance.embeddings
    )

    # Create reasoning chain visualizations
    reasoning_steps = create_reasoning_chain_visualization(
        search_instance,
        score_threshold
    )

    return final_graph, reasoning_steps

In [58]:
# Create visualizations
final_graph, reasoning_steps = display_graph_visualizations(search)

# Display figures (if using in a notebook)
final_graph.show()
for step_fig in reasoning_steps:
    step_fig.show()

AttributeError: 'TavilySearchResults' object has no attribute '_aggregate_results'

# Chunk retriever

In [6]:
class KnowledgeGraphChunkRetriever:
    def __init__(self, json_data: Dict):
        """
        Initialize the retriever with knowledge graph data.

        Args:
            json_data: Dictionary containing entities, relationships, and chunks
        """
        self.entities = [entity['entity_name'].strip() for entity in json_data['entities']]
        self.chunks = json_data['chunks']
        self.model = SentenceTransformer('all-MiniLM-L6-v2')

        # Create chunk embeddings
        self.chunk_embeddings = self.model.encode([chunk['content'] for chunk in self.chunks])

        # Create entity to chunk mapping
        self.entity_chunks = self._create_entity_chunk_mapping()

    def _create_entity_chunk_mapping(self) -> Dict[str, Set[int]]:
        """Create a mapping of entities to chunk indices where they appear."""
        entity_chunks = {entity: set() for entity in self.entities}

        for chunk_idx, chunk in enumerate(self.chunks):
            chunk_content = chunk['content'].lower()
            for entity in self.entities:
                if entity.lower() in chunk_content:
                    entity_chunks[entity].add(chunk_idx)

        return entity_chunks

    def get_relevant_chunks(self, ranked_entities: Dict[str, float],
                            max_chunks: int = 5,
                            similarity_threshold: float = 0.3) -> List[Dict]:
        """
        Retrieve relevant chunks based on ranked entities.

        Args:
            ranked_entities: Dictionary of entity names and their importance scores
            max_chunks: Maximum number of chunks to return
            similarity_threshold: Minimum similarity score threshold

        Returns:
            List of relevant chunks with their scores
        """
        # Collect chunk indices from top entities
        relevant_chunk_indices = set()
        for entity in ranked_entities:
            relevant_chunk_indices.update(self.entity_chunks.get(entity, set()))

        if not relevant_chunk_indices:
            return []

        # Calculate chunk scores
        chunk_scores = []
        for chunk_idx in relevant_chunk_indices:
            score = self._calculate_chunk_score(
                chunk_idx,
                ranked_entities,
                self.chunk_embeddings[chunk_idx]
            )
            if score >= similarity_threshold:
                chunk_scores.append({
                    'chunk': self.chunks[chunk_idx],
                    'score': score,
                    'relevant_entities': [
                        entity for entity in ranked_entities
                        if chunk_idx in self.entity_chunks[entity]
                    ]
                })

        # Sort and return top chunks
        chunk_scores.sort(key=lambda x: x['score'], reverse=True)
        return chunk_scores[:max_chunks]

    def _calculate_chunk_score(self,
                               chunk_idx: int,
                               ranked_entities: Dict[str, float],
                               chunk_embedding: np.ndarray) -> float:
        """Calculate relevance score for a chunk based on entities and content."""
        # Entity presence score
        entity_score = sum(
            ranked_entities[entity]
            for entity in ranked_entities
            if chunk_idx in self.entity_chunks[entity]
        )

        # Normalize entity score
        if entity_score > 0:
            entity_score = entity_score / max(ranked_entities.values())

        return entity_score


def process_query_with_chunks(json_data: Dict,
                              graph_search_results: Dict[str, float]) -> List[Dict]:
    """
    Process a query and return relevant chunks based on graph search results.

    Args:
        json_data: Knowledge graph data
        query_entities: List of query entity names
        graph_search_results: Results from graph search algorithm

    Returns:
        List of relevant chunks with scores and metadata
    """
    retriever = KnowledgeGraphChunkRetriever(json_data)
    return retriever.get_relevant_chunks(graph_search_results)

In [7]:
# Pydantic class definition
class FeedbackAgent(BaseModel):
    chunk_number:int=Field(description="chunk number of the provided dictionary")
    relevancy:bool=Field(description="whether the chunk is relevant to the question or not")
    feedback:list[str]=Field(description="feedbacks to the retrieved content.")

class Feedbacks(BaseModel):
    feedback:list[FeedbackAgent]=Field(description="List of feedbacks")

class QueryReformation(BaseModel):
    user_query:str=Field(description="original user question")
    new_query:str=Field(description="reformated question")

def search_kg(user_query, data):
    """
    search knowledge graph for extract relevent chunks

    :param user_query: user question
    :param data: json data of knowledge graph
    :return:
    """
    search = ChainOfThoughtGraphSearch(data)
    graph_results = search.iterative_search(user_query)

    # Then retrieve relevant chunks
    chunk_results = process_query_with_chunks(data, graph_results)
    # print(chunk_results)
    return chunk_results

def get_feedbacks(chunk_results, user_query):
    """This function is used to get the feedbacks to the retrived chunks to check whether the retrived chunks are relevant to the query or not"""
    #define empty variable for relevancy score
    rel_score = 0
    negative_feedback = [] # define empty list for collect feedbacks
    result = []

    #looping through the content
    for idx, item in enumerate(chunk_results, start=1):
        chunk_dict = {
            'chunk': idx,
            'content': item['chunk']['content']
        }
        result.append(chunk_dict)

    # define feedbacks structured output
    feedback_structured_output_model = llm.with_structured_output(schema=Feedbacks)

    feedback_list_from_llm = feedback_structured_output_model.invoke(
        [{"role":"system",
          "content":"You are a precise content relevance analyzer. Evaluate the relevance between the provided query and retrieved content chunks based on these criteria:\n\n1. Semantic relationship: Does the content directly address the query's main topic?\n2. Information value: Does the content provide useful information that helps answer the query?\n3. Context alignment: Is the content in the appropriate context of the query?\n\nOutput format:\n- If relevant: Return 'True'\n- If not relevant: Return 'False' followed by a JSON array of specific reasons why the content doesn't match the query's requirements\n\nExample reasons format:\n[\"Content discusses different topic than query\", \"Information is too general\", \"Context is inappropriate for the question\"]\n\nBe concise and decisive in your assessment."},
         {"role":"user",
          "content": f"Query: {user_query} \n Retrieved Content:{result}"}
          ])

    # check whether feedbacks are negative or not
    for element in feedback_list_from_llm.feedback:
        # print(element)
        if not element.relevancy:
            rel_score += 1
            negative_feedback.append(element)
    
    return rel_score, feedback_list_from_llm

    
def entity_set_formation():
    """This function is used to get all the entities from data and create an entities set with no duplicates."""
    entities_set = set()
    entities_list = []
    
    for entities in data['entities']:
        entities_list.append(entities['entity_name'])

    entities_set.update(entities_list)

    return entities_set


def question_reform(feedback_list_from_llm, user_query):
    """Reform the user query by using feedbacks received from the LLM and entities in the document corpus"""
    
    entity_set = entity_set_formation()
    feedback_list=[] # empty list for create one collection feedback string

    for element in feedback_list_from_llm.feedback:
    # print(element)
        if element.feedback:
            feedback_list.append(", ".join(element.feedback))

    feedbacks = "\n".join(feedback_list)

    # define new formatted query structured
    query_structured_output_model = model.with_structured_output(schema=QueryReformation)
    llm_remormated_query = query_structured_output_model.invoke(
        [{"role":"system",
          "content":"""You are a strict query optimization expert. Your mission is to enhance search queries while adhering to these mandatory requirements:

1. PRIMARY CONSTRAINTS:
- Generate queries using ONLY terms from the provided entity keyword set
- NEVER introduce concepts or terms not present in the entity keywords
- ANY deviation from the provided keywords will result in query rejection

2. ENHANCEMENT PROCESS:
- Start with the original query's core intent
- Match query concepts to available entity keywords
- Incorporate relevant feedback to avoid previous issues
- If a concept cannot be expressed using entity keywords, drop it
- If the query cannot be reformulated using available keywords, respond with 'QUERY NOT POSSIBLE'

3. FORMATTING RULES:
- Output format: single enhanced query string
- Maximum length: 2 sentences
- Must maintain grammatical correctness
- Include specific entity keywords verbatim

4. QUALITY CHECKS:
- Verify every term against entity keyword set
- Ensure semantic relationship to original query
- Confirm alignment with knowledge base feedback
- Validate that no external concepts are introduced

CRITICAL: If you cannot construct a meaningful query using ONLY the provided ENTITY KEYWORDS, output 'QUERY NOT POSSIBLE' instead of attempting to generate an invalid query beyond the entity keywords.
    """     
        },
         {"role":"user",
          "content": f"Original Query:{user_query}\nRetrieval Feedback:{feedbacks}\nAllowed Keywords:{entity_set}"}
          ])
    print(llm_remormated_query)
    return llm_remormated_query


# First perform graph search
def chain_of_thought_graphsearch(user_query, data):

    chunk_results = search_kg(user_query, data)
    rel_score, feedback_list_from_llm = get_feedbacks(chunk_results, user_query)
            
    if rel_score > 3:
        llm_remormated_query = question_reform(feedback_list_from_llm,user_query) # call query reformation function
        # print(reformatted_question.new_query)
        answer_chunks = search_kg(llm_remormated_query.new_query,data)
        
        # loop for only take the content from the answer chunk
        chunks_content =[]
        for contents in answer_chunks:
            chunks_content.append(contents.get('chunk').get('content'))
        
        #TavilySearch function calling
        search_result = tavily_search(llm_remormated_query.new_query)
        answer = answer_ranking(chunks_content, search_result,llm_remormated_query.new_query)
        
        
        #LLM call function for Final answer creation
        final_answer = final_answer_creation(llm_remormated_query.new_query,answer)
        
        save_qa_to_json(user_query,final_answer)
        print(f"Q&A pair successfully saved")
        # print(final_answer)
        
    else:
        search_result = tavily_search(user_query)

In [8]:
# create TavilySearch instance
search = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=True,
    # include_domains=[...],
    # exclude_domains=[...],
    # name="...",            # overwrite default tool name
    # description="...",     # overwrite default tool description
    # args_schema=...,       # overwrite default args_schema: BaseModel
)

def tavily_search(new_query):
    
    search_results = search.invoke({"query":new_query})
    content_list =[]
    for search_result in search_results:
        content_list.append(search_result['content'])
        
    return content_list

In [9]:
from rank_bm25 import BM25Okapi
import numpy as np

def answer_ranking(chunk_content, search_result, new_query):
    """Rank the answers based on the relevancy for the query."""
    chunk_content.extend(search_result)
    tokenized_corpus = [doc.split(" ") for doc in chunk_content]
    bm25 = BM25Okapi(tokenized_corpus)
    
    query=new_query.lower()
    tokenized_query = query.split(" ")
    doc_scores = bm25.get_scores(tokenized_query)
    result = bm25.get_top_n(tokenized_query, tokenized_corpus, n=len(chunk_content))
    
    return result

In [10]:
def final_answer_creation(new_query, answer):
    final_answer = model.invoke(
        [{"role":"system",
          "content":"""You are a precise answer synthesizer. Follow these requirements to generate accurate responses:

1. ERROR HANDLING:
- If user_query or User question equals 'QUERY NOT POSSIBLE', return exactly: 'Cannot generate an answer to your question, please update your question'
- If answer chunks don't contain relevant information, respond with 'INSUFFICIENT CONTEXT'

2. SYNTHESIS RULES:
- Use ONLY information present in the provided answer chunks
- Maintain factual accuracy without adding external knowledge
- Provide direct answers to the specific question asked

3. RESPONSE FORMAT:
- you should try to keep your answers short when possible otherwise concise answers maximum two sentence
- Use clear, simple language
- Maintain consistent tone throughout

4. QUALITY STANDARDS:
- Verify all information against provided chunks
- Ensure logical flow of information
- Avoid speculation or assumptions
- Present information in order of relevance
- If chunks contain conflicting information, acknowledge the conflict

5. PROHIBITED ACTIONS:
- No external knowledge incorporation
- No speculation beyond provided chunks
- No generalization without supporting evidence
- No answering questions outside chunk scope
- No combining unrelated information"""
        },
         {"role":"user",
          "content": f"User question:{new_query}\nanswer chunks:{answer}"}
          ])
    return final_answer.content   

In [11]:
def save_qa_to_json(query, answer, filename="qa_data.json"):
    """
    Creates a dictionary from a question and answer pair and saves it to a JSON file.
    
    Args:
        query (str): The question text
        answer (str): The answer text
        filename (str): Name of the output JSON file (default: 'qa_data.json')
        
    Returns:
        dict: The created question-answer dictionary
    """
    # Create the QA dictionary
    qa_dict = {
        "question": query,
        "answer": answer
    }
        # Check if the file already exists
    if os.path.exists(filename):
        # Read existing data
        try:
            with open(filename, 'r') as file:
                existing_data = json.load(file)
                
            # If the existing data is a list, append to it
            if isinstance(existing_data, list):
                existing_data.append(qa_dict)
                qa_data = existing_data
            # If it's a dictionary, convert to a list with both items
            else:
                qa_data = [existing_data, qa_dict]
        except json.JSONDecodeError:
            # If the file exists but isn't valid JSON, start fresh
            qa_data = [qa_dict]
    else:
        # If the file doesn't exist, create new data
        qa_data = [qa_dict]
    
    # Save the data to the JSON file
    with open(filename, 'w') as file:
        json.dump(qa_data, file, indent=4)
    
    print(f"Q&A pair successfully saved to {filename}")

In [12]:
def load_questions(file_path):
    """
    Load questions from a JSON file and return them as a list.
    
    Args:
        file_path (str): Path to the JSON file containing questions
        
    Returns:
        list: List of questions
    """
    try:
        with open(file_path, 'r') as file:
            content = file.read()
            # Split by newlines to get individual questions
            questions = [q for q in content.strip().split('\n') if q]
            return questions
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return []
    except json.JSONDecodeError:
        print(f"Error: File '{file_path}' is not valid JSON.")
        return []

In [13]:
questions = load_questions("inscit_query.json")
if questions:
    for i, question in enumerate(questions[2:4]):  # Limit to 10 questions
        print(f"\nQuestion {i+1}: {question}")
        chain_of_thought_graphsearch(user_query=question, data=data)
        


Question 1: When did people first start making cheese?
user_query='When did people first start making cheese?' new_query='history of traditional cheeses'
Q&A pair successfully saved to qa_data.json
Q&A pair successfully saved

Question 2: Do all cheeses need to be refrigerated?
user_query='Do all cheeses need to be refrigerated?' new_query='Do commercial cheeses and traditional cheeses require refrigeration?'
Q&A pair successfully saved to qa_data.json
Q&A pair successfully saved


In [12]:
chain_of_thought_graphsearch(user_query ="What are the effects of cheese on health?",data=data)

user_query='What are the effects of cheese on health?' new_query="What are the health effects of cheese made from cow's milk, goat's milk, or whole milk?"
Goat cheese is generally healthier than cow's milk cheese, offering benefits like easier digestion, lower calories, and less saturated fat. It also contains more medium-chain fatty acids and has a lower lactose content, making it suitable for those with lactose intolerance. Whole milk cheeses can be high in saturated fat, which may require moderation for individuals with cardiovascular concerns. Both types provide essential nutrients like calcium and protein, supporting bone and muscle health. 

Key differences:
- Goat cheese has A-2 beta-casein (less allergenic) vs. cow cheese's A-1 beta-casein
- Goat cheese may reduce inflammation and diabetes risk more effectively
- Whole milk cheeses offer higher fat content (beneficial for nutrient absorption but may need restriction in certain diets)
